In [19]:
import time
import psycopg2
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
import pandas as pd
import logging

#Loading the env
load_dotenv()

True

In [20]:
def time_until_next_1am():
    now = datetime.now()
    next_1am = now.replace(hour=1, minute=0, second=0, microsecond=0)
    if now >= next_1am:
        next_1am += timedelta(days=1)
    return (next_1am - now).total_seconds()

def setup_logger(name, log_file, level=logging.DEBUG):    
    # Create a custom logger
    logger = logging.getLogger(name)
    logger.setLevel(level)

    # Create handlers
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(level)

    # Create a formatter and set it for the handler
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(file_handler)

    return logger

def close_logger(logger):
    handlers = logger.handlers[:]
    for handler in handlers:
        handler.close()
        logger.removeHandler(handler)

In [21]:
def get_connection(logger):
    try:
        conn = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )
        logger.info("Connection established successfully!")
        
        return conn
    except Exception as e:
        print("Connection failed:", e)
        logger.error("Connection failed:", e)

In [22]:
def getAllPotholesToReport(logger):
    conn = get_connection(logger)

    cursor = conn.cursor()
    # Get today's date in YYYY-MM-DD format
    today = datetime.today().date()
    query = """
        SELECT pothole_id, pothole_size, description, address FROM potholes 
        WHERE is_reported = false
        """
    #Execute query with WHERE clause filtering for is_fixed=False and estimated_fix_date=today
    cursor.execute(query)

    # Fetch results (if applicable)
    rows = cursor.fetchall()

    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    cursor.close()
    conn.close()

    if df.empty:
        logger.info("No potholes received from DB.")
    else:
        logger.info("All potholes that need to reported retrieved successfully.")

    return df

In [34]:
def setToReported(successIDS, logger):
    conn = get_connection(logger)

    cursor = conn.cursor()

    update_query = "UPDATE potholes SET is_reported = true WHERE pothole_id = %s"

    #Pothole IDs to be updated
    data = [(pothole_id,) for pothole_id in successIDS]

    try:
        cursor.executemany(update_query, data)
        conn.commit()  
        print("Records updated successfully!")
        logger.info("Report Status updated successfully!")
    except Exception as e:
        print("Failed to update records:", e)
        logger.error("Failed to update records with Report Status:", e)
        conn.rollback() 

    cursor.close()
    conn.close()

In [37]:
logger = setup_logger('pothole_reporter', 'pothole_reporter.log')
df = getAllPotholesToReport(logger)
df.head()

,pothole_id,pothole_size,description,address
0,74,Zbbzz,Akakk,"208 Erb Street East, Waterloo, Ontario N2J 1M6..."
1,75,Timeeee,Chdcj time,"145 Lucan Avenue, Waterloo, Ontario N2J 1W6, C..."
2,79,Large,Ahahha,"300 weber st, Waterloo"
3,82,Djd,Djd,"256 willowdale place, Waterloo"
4,85,Large,Middle of St,"260 Lester St, Waterloo"


In [28]:
from scraper import submitForm

successIDS = []

for index, row in df.iterrows():
    result = submitForm(row['address'], row['description'], row['pothole_size'])
    if result:
        successIDS.append(row['pothole_id'])


Contact information (optional)


[48]

In [ ]:
setToReported(successIDS, logger)